In [ ]:
!pip install scikit-learn pandas numpy matplotlib plotly "nbformat>=4.2.0"

In [ ]:
!pip install ipykernel

In [ ]:
!pip install --upgrade nbformat

In [ ]:
from sklearn.datasets import make_classification
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.neural_network import MLPClassifier
import warnings
warnings.filterwarnings('ignore')

# generate random data
SAMPLES = 1000
x, y = make_classification(n_samples=SAMPLES, n_features=20, n_informative=15, random_state=42, n_classes=2)
# select 20% of the data for later testing
test_split_x = x[:int(SAMPLES * 0.2)]
test_split_y = y[:int(SAMPLES * 0.2)]
train_split_x = x[int(SAMPLES * 0.2):]
train_split_y = y[int(SAMPLES * 0.2):]

In [ ]:
labeled_data_selection_model = MLPClassifier(hidden_layer_sizes=(10), max_iter=50, random_state=42)
labeled_data_selection_model.fit(x, y)
scores = labeled_data_selection_model.predict_proba(train_split_x)
scores = scores[:, 1]  # get probabilities for the positive class

In [ ]:
# visualize the samples via pca:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca_x = pca.fit_transform(x)
pca_test_split_x = pca.transform(test_split_x)
pca_train_split_x = pca.transform(train_split_x)
fig = px.scatter(x=pca_x[:, 0], y=pca_x[:, 1], color=y, title="PCA of the data")
fig.add_trace(go.Scatter(x=pca_test_split_x[:, 0], y=pca_test_split_x[:, 1], mode='markers', name='Test Data', marker=dict(color='red', size=5, symbol='cross')))
fig.add_trace(go.Scatter(x=pca_train_split_x[:, 0], y=pca_train_split_x[:, 1], mode='markers', name='Train Data', marker=dict(color='blue', size=5, symbol='circle')))
fig.show()

In [ ]:
# barplot of the score distribution
fig = px.histogram(pd.Series(scores), nbins=50, title='Score Distribution', labels={'value': 'Score'})
fig.update_layout(xaxis_title='Score', yaxis_title='Count')
fig.show()

In [ ]:
# find threshold such that 5% of the data is above it
threshold_above = np.percentile(scores, 95)
threshold_below = np.percentile(scores, 5)
# filter data above the threshold
x_above = train_split_x[scores >= threshold_above]
y_above = train_split_y[scores >= threshold_above]
x_below = train_split_x[scores <= threshold_below]
y_below = train_split_y[scores <= threshold_below]

labeled_data_x = np.concatenate((x_above, x_below), axis=0)
labeled_data_y = np.concatenate((y_above, y_below), axis=0)

not_labeled_data_x = train_split_x[~np.isin(np.arange(len(train_split_x)), np.where(scores >= threshold_above)[0]) & ~np.isin(np.arange(len(train_split_x)), np.where(scores <= threshold_below)[0])]
not_labeled_data_y = train_split_y[~np.isin(np.arange(len(train_split_x)), np.where(scores >= threshold_above)[0]) & ~np.isin(np.arange(len(train_split_x)), np.where(scores <= threshold_below)[0])]

In [ ]:
# Train a propensity model:
propensity_model = MLPClassifier(hidden_layer_sizes=(10), max_iter=1000, random_state=42, early_stopping=True, validation_fraction=0.2, n_iter_no_change=10)
propensity_train_x = np.concatenate((labeled_data_x, not_labeled_data_x), axis=0)
propensity_train_y = np.concatenate((np.ones(len(labeled_data_x)), np.zeros(len(not_labeled_data_x))), axis=0)  # 1 for labeled, 0 for not labeled
propensity_model.fit(train_split_x, train_split_y)
# Predict propensity scores
propensity_scores = propensity_model.predict_proba(train_split_x)[:, 1]  # get propensities

In [ ]:
# visualize the embedding of the labeled data vs the unlabeled data
pca_labeled_data_x = pca.transform(labeled_data_x)
pca_not_labeled_data_x = pca.transform(not_labeled_data_x)
fig = px.scatter(x=pca_labeled_data_x[:, 0], y=pca_labeled_data_x[:, 1], color=labeled_data_y, title="PCA of the labeled data")
fig.add_trace(go.Scatter(x=pca_not_labeled_data_x[:, 0], y=pca_not_labeled_data_x[:, 1], mode='markers', name='Unlabeled Data', marker=dict(color='grey', size=5, symbol='x')))
fig.update_layout(xaxis_title='PCA Component 1', yaxis_title='PCA Component 2')
fig.show()

In [ ]:
train_data_labeled_x, test_data_labeled_x, train_data_labeled_y, test_data_labeled_y = train_test_split(labeled_data_x, labeled_data_y, test_size=0.4, random_state=42)

In [ ]:
# train a model on the labeled data
labeled_data_selection_model = MLPClassifier(hidden_layer_sizes=(10), max_iter=50, random_state=42, early_stopping=True, validation_fraction=0.2, n_iter_no_change=10)
labeled_data_selection_model.fit(train_data_labeled_x, train_data_labeled_y)
scores_outcome_model = labeled_data_selection_model.predict_proba(test_data_labeled_x)
print("ROC AUC Score (Labeled Data):", roc_auc_score(test_data_labeled_y, scores_outcome_model[:, 1]))
print("ROC AUC Score (all Data):", roc_auc_score(test_split_y, labeled_data_selection_model.predict_proba(test_split_x)[:, 1]))

In [ ]:
e_X = propensity_model.predict_proba(train_data_labeled_x)[:, 1]
e_X = np.clip(e_X, 1e-3, 1 - 1e-3)
T = labeled_data_selection_model.predict_proba(train_data_labeled_x)[:, 1]  # Assuming T is the treatment assignment, here we use the model's prediction as a proxy

# Schritt 2: Gewicht berechnen
weights = T / e_X + (1 - T) / (1 - e_X)

# Schritt 3: Outcome-Model trainieren mit gewichteter Regression
# z.B. sklearn: sample_weight=weights
new_outcome_model = MLPClassifier(hidden_layer_sizes=(10), max_iter=1000, random_state=42, early_stopping=True, validation_fraction=0.2, n_iter_no_change=10)
new_outcome_model.fit(np.hstack([train_data_labeled_x, T.reshape(-1, 1)]), train_data_labeled_y, sample_weight=weights)
# Predict the outcome using the new outcome model
scores_outcome_model = new_outcome_model.predict_proba(test_data_labeled_x)
print("ROC AUC Score (Labeled Data):", roc_auc_score(test_data_labeled_y, scores_outcome_model[:, 1]))
print("ROC AUC Score (all Data):", roc_auc_score(test_split_y, new_outcome_model.predict_proba(test_split_x)[:, 1]))

In [ ]:
from econml.dr import DRLearner


learner = DRLearner(model_propensity=propensity_model,
                    model_regression=labeled_data_selection_model)
learner.fit(Y, T, X)
tau_hat = learner.effect(X)